In [1]:
#Using Matminer Featurization to Run and Optimize a Algorithmia KSTAR Model
#
# Glen Joy
from matminer.utils.conversions import str_to_composition
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.base import BaseFeaturizer
from matminer.featurizers.composition import Stoichiometry
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import Algorithmia
# Authenticate with your API key
apiKey = "simr26nEeJguvzsmB8VIWPdeyo01"
# Create the Algorithmia client object
client = Algorithmia.client(apiKey)

In [3]:
# Viewing Dataset through Panda
pd.read_csv('../datasets/HydrogenStorageDataBasemagpie.data.csv')

,comp,H2_wtpercent,Enthalpy,Temp_C,P_atm,Phase
0,Th1.5Ce.5Al,0.40,133.00,650,0.000300,A2B
1,Th2Al,0.80,130.00,500,0.001000,A2B
2,Ti2Cu,2.20,130.00,500,0.120000,A2B
3,Ti2Pd,1.40,90.00,0,1000.000000,A2B
4,Zr2Cu,1.30,144.00,600,0.003000,A2B
5,Zr2Cu,1.30,142.00,700,0.020000,A2B
6,Zr2Ni,1.30,183.00,604,0.003000,A2B
7,Hf2Fe,1.10,70.00,277,0.380000,A2B
8,Mg1.5Ni,2.60,29.10,300,3.700000,A2B
9,Mg1.75Ni,2.80,26.60,300,4.500000,A2B


In [4]:
# Instantiate a DataDirectory object, set your data URI and call create
nlp_directory = client.dir("data://glenjoy99/nlp_directory")
# Create your data collection if it does not exist
if nlp_directory.exists() is False:
	nlp_directory.create()

In [5]:
# Imporing and storing dataset into DataFrame object
df= pd.read_csv('../datasets/HydrogenStorageDataBasemagpie.data.csv')
# Allowing composition names in the dataset to be in  a recognizable format 
df["composition"] = df["comp"].transform(str_to_composition)
# Applying Magpie preset ELEMENT PROPERTY attributes for each composition
ep_feat = ElementProperty.from_preset(preset_name="magpie")
df = ep_feat.featurize_dataframe(df, col_id="composition")
        # Applying Magpie preset BASE FEATURE attributes for each composition
        #bf_feat = BaseFeaturizer.from_preset(preset_name="magpie")
        #df = bf_feat.featurize_dataframe(df, col_id="composition")
# Setting target output value to Enthalpy
y = df['Enthalpy'].values
# Exluding our output value and all non-numeric values from our input
excluded = ["Enthalpy", "Phase", "comp", "composition"]
X = df.drop(excluded, axis =1)

In [6]:
# Converts and saves to CSV, later converted to arff
# df.to_csv('featurizedDataset.csv')

In [7]:
# Creates variable to store path to featurized dataset on Algorithmia
database_file = "data://glenjoy99/nlp_directory/featurizedDataset.arff"

In [8]:
# Puts featurized dataset in the directory on Algorithmia
client.file(database_file).putFile("featurizedDataset.arff")

In [9]:
# Downloads the file and saves as localfile 
if client.file(database_file).exists() is True:
	localfile = client.file(database_file).getFile()

In [10]:
# Download contents of file as a string and stores into input variable
if client.file(database_file).exists() is True:
	input = client.file(database_file).getString()

In [11]:
# Lists the added Magpie attributes/descriptors 
df.head()
print("There are {} possible descriptors:\n\n{}".format(X.shape[1], X.columns.values))

There are 135 possible descriptors:

['H2_wtpercent' 'Temp_C' 'P_atm' 'minimum Number' 'maximum Number'
 'range Number' 'mean Number' 'avg_dev Number' 'mode Number'
 'minimum MendeleevNumber' 'maximum MendeleevNumber'
 'range MendeleevNumber' 'mean MendeleevNumber' 'avg_dev MendeleevNumber'
 'mode MendeleevNumber' 'minimum AtomicWeight' 'maximum AtomicWeight'
 'range AtomicWeight' 'mean AtomicWeight' 'avg_dev AtomicWeight'
 'mode AtomicWeight' 'minimum MeltingT' 'maximum MeltingT'
 'range MeltingT' 'mean MeltingT' 'avg_dev MeltingT' 'mode MeltingT'
 'minimum Column' 'maximum Column' 'range Column' 'mean Column'
 'avg_dev Column' 'mode Column' 'minimum Row' 'maximum Row' 'range Row'
 'mean Row' 'avg_dev Row' 'mode Row' 'minimum CovalentRadius'
 'maximum CovalentRadius' 'range CovalentRadius' 'mean CovalentRadius'
 'avg_dev CovalentRadius' 'mode CovalentRadius'
 'minimum Electronegativity' 'maximum Electronegativity'
 'range Electronegativity' 'mean Electronegativity'
 'avg_dev Electrone

In [12]:
# Creates inputs for KSTAR algorithm
input = {
  "trainUrl": "data://glenjoy99/nlp_directory/featurizedDataset.arff",
  "cv": 10,
  "options": "",
  "mode": "train",
  "modelUrl": "data://.algo/temp/model.txt"
}
# Sets client Algorithma API key 
# client = Algorithmia.client('simr26nEeJguvzsmB8VIWPdeyo01')
# Sets model object to Weka KSTAR model 
algo = client.algo('weka/KStar/0.1.1')
# Pipes input into algorithm and prints results
print(algo.pipe(input))

AlgorithmException: 'java.lang.NullPointerException'